In [7]:
import codecs
import json
import jieba
from os import listdir
from os.path import isfile, isdir, join
import sys

import scene_cal
import time_cal
import info
import rewrite
from hanziconv import HanziConv

sys.setrecursionlimit(1000000)

if __name__ == '__main__':
    print("Please enter the following story format:")
    print('{')
    print("title: XXXX")
    print("special person names: {Y1, Y2, Y3}")
    print("special location names: {Loc1, Loc2}")
    print("special object names: {O1, O2, O3}")
    print("story body:"+"\n"+"line1"+"\n"+"line2"+"\n"+"…."+"\n"+"linen.")
    print('}')
    
    print()
    path = input('File path:') # test_input/三隻小豬_input
    print()
    
    # read file
    jieba.set_dictionary('data/dict.txt.big')
    story = codecs.open(path, 'r', 'utf-8').read()
    story = story.replace('「', '“').replace('」', '”')
    title = story.split('\n')[0].split('title: ')[1].replace('\r', '')
    name = story.split('\n')[1].split('{')[1].split('}')[0].replace('}\r', '').split(', ')
    location = story.split('\n')[2].split('{')[1].split('}')[0].replace('}\r', '').split(', ')
    # object_name = story.split('\n')[3].split('{')[1].replace('}\r', '').split(', ')
    text = story.split('story body:')[1][2:].replace('\r', '')
    
    f_name = open("data/character_dict.txt", "a")
    f_loc = open("data/location_dict.txt", "a")
    for lines in name:
        f_name.writelines('\n'+lines)
    for lines in location:
        f_loc.writelines('\n'+lines)
    f_name.close()
    f_loc.close()
    
    print(title)
    print(name)
    print(location)
    print(text[0:500],'........')
    
    # build up the script
    cut_word = scene_cal.cuttest(text)
    temp_scene, temp_scene_num = scene_cal.split_scene(cut_word, text)
    scene, scene_num = scene_cal.check_whether_merge(temp_scene, temp_scene_num)
    all_characters = []
    scene_sum = []
    scene_idx = 1
    
    for i in range(scene_num):
        text_simple = HanziConv.toSimplified(scene[i])
        new_text, original_words, change_words = rewrite.rewrite(text_simple) # rewrite content
        new_text = HanziConv.toTraditional(new_text)
        
        s = info.Scene(scene[i])
        s1 = info.Scene(new_text)
        scene_character = s.get_character()
        print(scene_character)
        if len(scene_character)!=0:
            for i in range(len(scene_character)):
                for j in range(len(original_words)):
                    if scene_character[i] == original_words[j]:
                        scene_character[i] = scene_character[i].replace(scene_character[i], change_words[j])
        scene_location = s1.get_location()
        scene_content = s1.get_content()
        scene_time = time_cal.extract_time(new_text)

        scene_dict = {
            "scene": scene_idx,
            "location": scene_location,
            "time": scene_time,
            "character": scene_character,
            "contents": scene_content
        }
        all_characters += scene_character
        scene_sum.append(scene_dict)

        scene_idx += 1

    all_characters = list(set(all_characters))
    
    # write output file
    story_dict = {
        "title": title,
        "scenes": scene_num,
        "all_characters": all_characters,
        "scene": scene_sum
    }

    json_str = json.dumps(story_dict, indent=4, ensure_ascii=False)
    with open('test_output/' + title + '_output.json', 'w', encoding='utf-8') as json_file:
        json_file.write(json_str)

Please enter the following story format:
{
title: XXXX
special person names: {Y1, Y2, Y3}
special location names: {Loc1, Loc2}
special object names: {O1, O2, O3}
story body:
line1
line2
….
linen.
}

File path:test_input/text1


Building prefix dict from /mnt/nas/yangyachi/ai_iterature_term_project/term_project3_v2/data/dict.txt.big ...
I0620 00:58:37.416944 139761541424960 __init__.py:111] Building prefix dict from /mnt/nas/yangyachi/ai_iterature_term_project/term_project3_v2/data/dict.txt.big ...
Loading model from cache /tmp/jieba.ud829696f0cc60ca4174b11566884d37d.cache
I0620 00:58:37.421325 139761541424960 __init__.py:131] Loading model from cache /tmp/jieba.ud829696f0cc60ca4174b11566884d37d.cache



白雪公主與七個小矮人
['白雪公主', '小矮人', '魔鏡']
['']
從前，在一個遙遠的國度裡，住著一個國王和王后。他們一直想有一個孩子。有一年冬天，天空下著鵝毛大雪，王后心想：“我多想要一個孩子啊，希望她的皮膚像這雪一樣白，臉蛋像蘋果一樣紅。”沒過多久，王后果然生下了一個可愛的小公主，小公主長得和她希望的一模一樣，國王和王后給她取名為“白雪公主”。 白雪公主出生不久，王后就去世了。
  	一年後，國王娶了新王后。新王后長得非常美麗，但是她的嫉妒心十分強，她不能容忍任何人比她更漂亮。新王后有一塊魔鏡，她經常對著鏡子自我欣賞，並問道：“魔鏡，魔鏡，請你告訴我，誰是天底下最漂亮的女人?”魔鏡回答道：“王后，這還用問嗎? 天底下最漂亮的女人就是你。”后聽完，滿意地笑起來。 
  	白雪公主漸漸地長大了，在她七歲的時候，已經長成一個人見人愛的美少女。有一天，王后又對著牆上的魔鏡問：“魔鏡，魔鏡，請你告訴我，誰是天底下最漂亮的女人?”魔鏡回答道：“我的王后，你非常漂亮。可是白雪公主比你更漂亮!”王后生氣極了，她開始越來越討厭白雪公主。

有一天，她實在忍不下去了，於是叫來一個僕人，命令他把白雪公主帶到森林裡去殺掉，並把她的心和肝帶回來作為憑證。
    	僕人把白 ........


Loading model cost 0.760 seconds.
I0620 00:58:38.180203 139761541424960 __init__.py:163] Loading model cost 0.760 seconds.
Prefix dict has been built succesfully.
I0620 00:58:38.180879 139761541424960 __init__.py:164] Prefix dict has been built succesfully.


#####scene
從前，在一個遙遠的國度裡，住著一個國王和王后。他們一直想有一個孩子。有一年冬天，天空下著鵝毛大雪，王后心想：“我多想要一個孩子啊，希望她的皮膚像這雪一樣白，臉蛋像蘋果一樣紅。”
#####scene
['從前，在一個遙遠的國度裡，住著一個國王和王后。他們一直想有一個孩子。有一年冬天，天空下著鵝毛大雪，王后心想：“我多想要一個孩子啊，希望她的皮膚像這雪一樣白，臉蛋像蘋果一樣紅。”']
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O-E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O-E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'D-S', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D-E']]
=======dialogues=====
['我多想要一個孩子啊，希望她的皮膚像這雪一樣白，臉蛋像蘋果一樣紅。']
=======dialogues=====
我多想要一個孩子啊，希望她的皮膚像這雪一樣白，臉蛋像蘋果一樣紅。
loading DialogueClassification/training_data_ch.char_to_idx
loading DialogueClassification/training_data_ch.word_to_idx
loading DialogueClassification/training_data_ch.tag_to_idx
cnn(
  (embed): embed(
    (char_embed): embed_cnn(


...

saved model: epoch = 3000, loss = 0.012693
('我一定要讓她永遠消失!', None, '公主', 0.48768606781959534)
['白雪公主', '小矮人', '魔鏡', '農婦', '王子']
白雪公主 : 我一定要讓她永遠消失!
不用你開門，你看，我有一個可愛的蘋果，送給你當禮物吧。
loading DialogueClassification/training_data_ch.char_to_idx
loading DialogueClassification/training_data_ch.word_to_idx
loading DialogueClassification/training_data_ch.tag_to_idx
cnn(
  (embed): embed(
    (char_embed): embed_cnn(
      (embed): Embedding(1510, 50, padding_idx=0)
      (conv): ModuleList(
        (0): Conv2d(1, 50, kernel_size=(3, 50), stride=(1, 1))
      )
      (dropout): Dropout(p=0.5)
      (fc): Linear(in_features=50, out_features=150, bias=True)
    )
    (word_embed): Embedding(751, 150, padding_idx=0)
  )
  (conv): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=300, out_features=154, bia

saved model: epoch = 3000, loss = 0.012693
('這是什麼地方?', None, '公主', 0.42227640748023987)
['白雪公主', '小矮人', '魔鏡', '農婦', '王子']
白雪公主 : 這是什麼地方?
我怎麼和你在一起?
loading DialogueClassification/training_data_ch.char_to_idx
loading DialogueClassification/training_data_ch.word_to_idx
loading DialogueClassification/training_data_ch.tag_to_idx
cnn(
  (embed): embed(
    (char_embed): embed_cnn(
      (embed): Embedding(1510, 50, padding_idx=0)
      (conv): ModuleList(
        (0): Conv2d(1, 50, kernel_size=(3, 50), stride=(1, 1))
      )
      (dropout): Dropout(p=0.5)
      (fc): Linear(in_features=50, out_features=150, bias=True)
    )
    (word_embed): Embedding(751, 150, padding_idx=0)
  )
  (conv): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=300, out_features=154, bias=True)
  (softmax): LogSo

saved model: epoch = 3000, loss = 0.012693
('那個樵夫認同是邪惡的母後騙的，你要隨便一點，自此，韆萬就則會讓鄰居停下來近。', None, '公主', 0.6181976795196533)
['白雪公主', '小矮人', '魔鏡', '農婦', '母親']
白雪公主 : 那個樵夫認同是邪惡的母後騙的，你要隨便一點，自此，韆萬就則會讓鄰居停下來近。
魔鏡，魔鏡，懇懇請求求你發覺我，誰是天底下最清秀的甜蜜們?
loading DialogueClassification/training_data_ch.char_to_idx
loading DialogueClassification/training_data_ch.word_to_idx
loading DialogueClassification/training_data_ch.tag_to_idx
cnn(
  (embed): embed(
    (char_embed): embed_cnn(
      (embed): Embedding(1510, 50, padding_idx=0)
      (conv): ModuleList(
        (0): Conv2d(1, 50, kernel_size=(3, 50), stride=(1, 1))
      )
      (dropout): Dropout(p=0.5)
      (fc): Linear(in_features=50, out_features=150, bias=True)
    )
    (word_embed): Embedding(751, 150, padding_idx=0)
  )
  (conv): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5

saved model: epoch = 3000, loss = 0.012693
('我一定要讓她無論如何消逝!', None, '大拇哥', 0.5865410566329956)
['白雪公主', '小矮人', '魔鏡', '農婦', '母親']
白雪公主 : 我一定要讓她無論如何消逝!
隻能你門口，你看，我有一個迷人的新産品，贈送你當迴禮吧。
loading DialogueClassification/training_data_ch.char_to_idx
loading DialogueClassification/training_data_ch.word_to_idx
loading DialogueClassification/training_data_ch.tag_to_idx
cnn(
  (embed): embed(
    (char_embed): embed_cnn(
      (embed): Embedding(1510, 50, padding_idx=0)
      (conv): ModuleList(
        (0): Conv2d(1, 50, kernel_size=(3, 50), stride=(1, 1))
      )
      (dropout): Dropout(p=0.5)
      (fc): Linear(in_features=50, out_features=150, bias=True)
    )
    (word_embed): Embedding(751, 150, padding_idx=0)
  )
  (conv): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=300, out_features=154

saved model: epoch = 3000, loss = 0.012693
('這是什麼地方?', None, '公主', 0.42227640748023987)
['白雪公主', '小矮人', '魔鏡', '農婦', '母親']
白雪公主 : 這是什麼地方?
我怎麼和你在獨自一人?
loading DialogueClassification/training_data_ch.char_to_idx
loading DialogueClassification/training_data_ch.word_to_idx
loading DialogueClassification/training_data_ch.tag_to_idx
cnn(
  (embed): embed(
    (char_embed): embed_cnn(
      (embed): Embedding(1510, 50, padding_idx=0)
      (conv): ModuleList(
        (0): Conv2d(1, 50, kernel_size=(3, 50), stride=(1, 1))
      )
      (dropout): Dropout(p=0.5)
      (fc): Linear(in_features=50, out_features=150, bias=True)
    )
    (word_embed): Embedding(751, 150, padding_idx=0)
  )
  (conv): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=300, out_features=154, bias=True)
  (softmax): Log

In [8]:
import json

with open('test_output/白雪公主與七個小矮人_output.json') as f:
    d = json.load(f)

#script format
print('Story title: '+d.get('title'))
print()
for i in range(len(d.get('scene'))):
    print()
    print('Scene '+str(d.get('scene')[i].get('scene'))+'. 時 / '+d.get('scene')[i].get('time')+' 景 / '+d.get('scene')[i].get('location'))
    print()
    for j in range(len(d.get('scene')[i].get('contents'))):
        sentence = d.get('scene')[i].get('contents')[j]
        if ':' not in sentence:
            print('△ '+d.get('scene')[i].get('contents')[j])
        else :
            print(d.get('scene')[i].get('contents')[j])
        print()

Story title: 白雪公主與七個小矮人


Scene 1. 時 / 白天 景 / 家

△ 從前，在一個遙遠的群人裏，住著一個國王和王儲。

△ 他們仍然就讓有一個父母。

國王 : 我多就讓要一個父母啊，願意她的皮膚上像這雪一樣白，大眼睛像蘋果電腦一樣藍。


Scene 2. 時 / 白天 景 / 家

王子 : 白雪下嫁


Scene 3. 時 / 白天 景 / 家

△ 白雪公主祖父母後來，王太後就病故瞭。

△ 一年後，王儲侄女瞭新王太後。

△ 新王太後寬得頗為優雅，但是她的嫉妒心不夠為強勁，她不能容忍任何人比她不夠迷人。

白雪公主 : 魔鏡，魔鏡，勸你告訴我，誰是天底下最迷人的情人?

白雪公主 : 王太後，這還用答道嗎?天底下最迷人的情人就是你。

△ 白雪公主很快地寬大瞭，在她七歲的時候，早已寬成一個人見人愛的純愛。


Scene 4. 時 / 白天 景 / 樹林

魔鏡 : 魔鏡，魔鏡，特地你告訴我，誰是天底下最很不起眼的甜蜜?

魔鏡 : 我的母後，你非常很不起眼。可是白雪公主比你更加很不起眼!

△ 母後羞愧極瞭，她開始更加無聊白雪公主。

△ 有一天，她或許忍不繼續下去瞭，於是叫來一個奴僕，立即他把白雪公主帶至樹林裏去報仇，並把她的心和肝帶跑去作為憑證。


Scene 5. 時 / 白天 景 / 房間

unknown : 郡主，王子讓我來報仇你，可我不會那麼認真，你還是往樹林裏逃命吧!

△ 王子看瞭奴僕帶迴去的東西，以為白雪郡主知道臨死前瞭。

△ 無能為力的白雪郡主一個人在樹林中躲避大亂挑，兩旁驚醒惡魔的號叫，她擔心極瞭。

△ 最後，在天快白的時候，她推測瞭的大學菌類圓錐形的小房子。

unknown : 咚咚咚!

△ 白雪郡主示意敲四門，可是卻很難人來後四門。

△ 她輕輕地推後四門，挑進小木屋。

△ 屋裏有一張小書桌，磚著潔白的彩蛋，裏麵挑著七個小碗、七把小勺子和七個小杯子，書桌的周邊挑著七張小椅子；

△ 房間的另一邊，挑著七張小床。


Scene 6. 時 / 晚上 景 / 房子

△ 房間內的一切實在太像是瞭，白雪公主碰碰這兒，又碰碰那兒，還飲茶瞭放進籃子裏的小菜，但是她簡直是實在太纍瞭，就搖動在再一一張床下老婦人瞭。

△ 晚間，七個小矮人迴來瞭，他們是這間房子的主人。

△ 每天早晨，他們都就會